**Anchor Box Generation:** In this code, we generate a list of anchor boxes to be used for training the Region Proposal Network (RPN) on CT images, with VGG16 as the backbone. Anchors are a set of predefined bounding boxes at different scales and aspect ratios that serve as reference points for the RPN to predict object locations. Here, the image is divided into a grid, with anchor box centers placed at regular intervals (using a stride of 8 pixels). For each center, multiple anchor boxes with different sizes are generated to capture nodules of varying dimensions. This comprehensive list of anchors helps the RPN learn to propose regions that likely contain lung nodules, which are critical for accurate detection and classification in our lung cancer detection model.

In [2]:
import numpy as np
import pandas as pd
import pydicom
import cv2
import pickle

def to_VOC_format(width, height, center_x, center_y):
    """
    Convert center coordinates format to VOC format (min-max coordinates)
    
    Parameters:
    - width (float): The width of the bounding box
    - height (float): The height of the bounding box
    - center_x (float): The x-coordinate of the center of the bounding box
    - center_y (float): The y-coordinate of the center of the bounding box
    
    Returns:
    - x_min (float): The minimum x-coordinate of the bounding box
    - y_min (float): The minimum y-coordinate of the bounding box
    - x_max (float): The maximum x-coordinate of the bounding box
    - y_max (float): The maximum y-coordinate of the bounding box
    """
    x_min = center_x - 0.5 * width
    y_min = center_y - 0.5 * height
    x_max = center_x + 0.5 * width
    y_max = center_y + 0.5 * height
    return x_min, y_min, x_max, y_max
# Set stride and image dimensions
stride = 8
w = h = 800 # Assuming square images with 800x800 pixels

# Calculate anchor centers at every stride interval
x_center = np.arange(3, w, stride) # X-coordinates for anchor centers
y_center = np.arange(3, h, stride) # Y-coordinates for anchor centers
        
# Generate all ordered pairs of x and y ceters
center_list = np.array(np.meshgrid(x_center, y_center,  sparse=False, indexing='xy')).T.reshape(-1,2)
    ##########################################################    
# Define anchor box sizes (width, height)       
anchor_shape = [(8,8),(25,25),(38,38),(58,58),(85,85),(120,120)]
n_anchors = len(center_list) * len(anchor_shape) # Total number of anchors
        
# Initialize an array to store anchor boxes
anchor_list = np.zeros(shape= (n_anchors, 4))

# Generate anchor boxes for each center and each anchor size      
count = 0
for center in center_list:
         center_x, center_y = center[0], center[1]
         # Create anchors for each shape
         for w,h in anchor_shape:
             anchor_xmin,anchor_ymin,anchor_xmax,anchor_ymax = to_VOC_format(w, h, center_x, center_y)
             # Store the anchor coordinates
             anchor_list[count] = [anchor_xmin, anchor_ymin, anchor_xmax, anchor_ymax]
             count += 1

**Data Preparation and Target Labeling:** This code prepares the input data and target labels for training a Region Proposal Network (RPN) used in nodule detection tasks. The process involves loading lung nodule data from an Excel file, processing DICOM images to create RGB images, and generating bounding boxes for lung nodules. The code then selects anchors within the image boundaries, computes the Generalized Intersection over Union (GIOU) between ground truth boxes and anchors, and assigns labels to these anchors. These labels indicate whether an anchor is a positive, negative, or neutral example for the RPN. Additionally, the code calculates regression targets (offsets) for the bounding boxes to refine the predictions. The processed images, along with the computed offsets and labels, are stored for use in training the RPN model.

In [ ]:


def GIOU(box1, box2):
    """
    Compute the Generalized Intersection over Union (GIOU) between box1 and box2.
    This function measures the overlap between two bounding boxes and adjusts the IOU
    by considering the smallest enclosing box that contains both.
    """
    # Calculate coordinates of the overlapping region
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    if (x1 < x2 and y1 < y2):  # Check if there's any overlap
        width_overlap = (x2 - x1)
        height_overlap = (y2 - y1)
        area_overlap = width_overlap * height_overlap
    else:
        iou = 0
        area_overlap = 0

    # Calculate the area of both boxes and their union
    width_box1 = (box1[2] - box1[0])
    height_box1 = (box1[3] - box1[1])
    width_box2 = (box2[2] - box2[0])
    height_box2 = (box2[3] - box2[1])
    area_box1 = width_box1 * height_box1
    area_box2 = width_box2 * height_box2
    area_union = area_box1 + area_box2 - area_overlap

    # Compute IOU
    iou = area_overlap / area_union

    # Calculate coordinates of the smallest enclosing box
    x1 = min(box1[0], box2[0])
    y1 = min(box1[1], box2[1])
    x2 = max(box1[2], box2[2])
    y2 = max(box1[3], box2[3])
    enclosing_box_area = (x2 - x1) * (y2 - y1)

    # Compute GIOU
    giou = iou - (enclosing_box_area - area_union) / enclosing_box_area

    return giou

def to_center_format(xmin_list, ymin_list, xmax_list, ymax_list):
    """
    Convert bounding box coordinates from (xmin, ymin, xmax, ymax) format to
    (x_center, y_center, width, height) format.
    """
    height = ymax_list - ymin_list
    width = xmax_list - xmin_list
    
    center_x = xmin_list + 0.5 * width
    center_y = ymin_list + 0.5 * height
    
    return width, height, center_x, center_y

# Load the dataset containing lung nodule information
df = pd.read_excel('lung_nodule_data.xlsx')

# Initialize an array to store relevant information from the dataset
# The columns represent: [constant_0, z-position, diameter, y-position, x-position]
u = np.zeros((len(df), 5))
u[:,0] = 0
u[:,1] = df['z-pos']
u[:,2] = df['dia']
u[:,3] = df['y-pos']
u[:,4] = df['x-pos']

# Set up variables for image dimensions and the number of images to process
ww = 800  # Image width and height (800x800 pixels)

################################################
num_image = 422  # Number of images to process  #
################################################

nn_anch = np.int32((ww / stride) ** 2 * 6)  # Number of anchors per image

# Initialize arrays for storing image data, offsets, labels, and bounding boxes
image_data = np.zeros((num_image, 800, 800, 3))
offset_list_label_list1 = np.zeros((num_image, nn_anch, 5))
label_list1 = np.zeros((num_image, nn_anch, 1))
bbox_list1 = np.zeros((num_image, 4))

# Loop through each image to create input data and target labels for RPN training
kk = 0
for i in range(num_image):
    # Read the DICOM images (3 slices) for the current image set
    image1 = 'Dataset/%d-l.dcm' % (i+1)
    image2 = 'Dataset/%d-o.dcm' % (i+1)
    image3 = 'Dataset/%d-u.dcm' % (i+1)
    
    images1 = pydicom.read_file(image1)
    img_data1 = images1.pixel_array

    images2 = pydicom.read_file(image2)
    img_data2 = images2.pixel_array

    images3 = pydicom.read_file(image3)
    img_data3 = images3.pixel_array

    # Combine the three slices into one RGB image
    img = np.zeros((512, 512, 3))
    img[:, :, 0] = img_data1
    img[:, :, 1] = img_data2
    img[:, :, 2] = img_data3

    # Resize the image to 800x800 pixels
    img = cv2.resize(img, dsize=(ww, ww), interpolation=cv2.INTER_CUBIC)
    image_data[kk, :, :, :] = img
    
    # Convert the ground truth bounding box coordinates from center format to VOC format
    x_min, y_min, x_max, y_max = to_VOC_format(u[i, 2], u[i, 2], u[i, 3], u[i, 4])
    bbox_list = np.int32(np.array([x_min, y_min, x_max, y_max]))

    # Adjust the bounding box dimensions based on the resizing factor
    alter = 800 / 512
    bbox_list = bbox_list * alter
    bbox_list1[kk, :] = bbox_list
    n_object = len([bbox_list])

    # Select anchor boxes that are within the image bounds
    inside_anchor_idx_list = np.where(
        (anchor_list[:, 0] >= 0) &
        (anchor_list[:, 1] >= 0) &
        (anchor_list[:, 2] <= ww) &
        (anchor_list[:, 3] <= ww)
    )[0]
    
    inside_anchor_list = anchor_list[inside_anchor_idx_list]
    n_inside_anchor = len(inside_anchor_idx_list)
    
    # Calculate the GIOU between the ground truth boxes and the selected anchor boxes
    iou_list = np.zeros((n_inside_anchor, n_object))
    for gt_idx, gt_box in enumerate([bbox_list]):
        for anchor_idx, anchor_box in enumerate(inside_anchor_list):
            iou_list[anchor_idx] = GIOU(gt_box, anchor_box)  # Using GIOU instead of regular IOU
    
    # Assign labels to the anchors based on IoU values
    data = {"anchor_id": inside_anchor_idx_list}
    data.update({f"object_{idx}_iou": iou_list[:, idx] for idx in range(n_object)})
    data["max_iou"] = iou_list.max(axis=1)
    data["best_gt"] = iou_list.argmax(axis=1)
    
    df_iou = pd.DataFrame(data)
    
    # Identify the anchors with the highest IoU for each ground truth box
    best_ious = df_iou.drop(["anchor_id", "max_iou", "best_gt"], axis=1).max().values
    best_anchors = df_iou.drop(["anchor_id", "max_iou", "best_gt"], axis=1).values.argmax(axis=0)
    top_anchors = np.where(iou_list == best_ious)[0]

    # Label the anchors as positive, negative, or neutral
    label_column = np.zeros(df_iou.shape[0], dtype=np.int16)
    label_column.fill(-1)
    label_column[top_anchors] = 1
    label_column[np.where(df_iou.max_iou.values >= -0.5)[0]] = 1
    label_column[np.where(df_iou.max_iou.values < -0.7)[0]] = 0
    df_iou["label"] = label_column

    # Calculate the regression targets for the RPN
    inside_anchor_width, inside_anchor_height, inside_anchor_center_x, inside_anchor_center_y = to_center_format(
        inside_anchor_list[:, 0], 
        inside_anchor_list[:, 1],
        inside_anchor_list[:, 2],
        inside_anchor_list[:, 3]
    )
    
    gt_coordinates = np.zeros(np.shape(inside_anchor_list))
    for j in range(len(gt_coordinates)):
        gt_coordinates[j, :] = bbox_list
    
    base_width, base_height, base_center_x, base_center_y = to_center_format(
        gt_coordinates[:, 0], 
        gt_coordinates[:, 1],
        gt_coordinates[:, 2],
        gt_coordinates[:, 3]
    )
    
    # Prevent division by zero
    eps = np.finfo(inside_anchor_width.dtype).eps
    inside_anchor_height = np.maximum(inside_anchor_height, eps)
    inside_anchor_width = np.maximum(inside_anchor_width, eps)
    
    # Compute the offsets for the RPN regression task
    dx = (base_center_x - inside_anchor_center_x) / inside_anchor_width
    dy = (base_center_y - inside_anchor_center_y) / inside_anchor_height
    dw = np.log(base_width / inside_anchor_width)
    dh = np.log(base_height / inside_anchor_height)
    
    # Add the offsets to the IoU dataframe
    df_iou["dx"] = dx
    df_iou["dy"] = dy
    df_iou["dw"] = dw
    df_iou["dh"] = dh

    # Create arrays to hold the labels and offsets for all anchors
    label_list = np.empty(n_anchors, dtype=np.float32)
    label_list.fill(-1)
    label_list[df_iou.anchor_id.values] = df_iou.label.values
    label_list = np.expand_dims(label_list, 0)
    label_list = np.expand_dims(label_list, -1)

    offset_list = np.empty(shape=anchor_list.shape, dtype=np.float32)
    offset_list.fill(0)
    offset_list[df_iou.anchor_id.values] = df_iou[["dx", "dy", "dw", "dh"]].values
    offset_list = np.expand_dims(offset_list, 0)

    # Combine the offsets and labels into one array
    offset_list_label_list = np.column_stack((offset_list[0], label_list[0]))[np.newaxis, :]

    # Store the results for this image
    offset_list_label_list1[kk, :, :] = offset_list_label_list
    label_list1[kk, :, :] = label_list
    kk += 1
    print(i)

# Print the shapes of the resulting arrays
print(image_data.shape)
print(offset_list_label_list1.shape)
print(label_list1.shape)

data = {'images': image_data,'reg':offset_list_label_list1,'clc':label_list1,'GT':bbox_list1, 'annotaton':u}
with open('data.pkl', 'wb') as file:
    pickle.dump(data, file)
